# 網頁爬取與 RAG 整合

本範例展示：
1. **第1個儲存格**：使用 WebBaseLoader 爬取網頁並建立向量資料庫
2. **第2個儲存格**：查詢網頁內容並展示 RAG 應用

學習目標：理解如何將網頁內容整合到 RAG 系統中，實現動態知識檢索

In [ ]:
# 第1個儲存格：使用 WebBaseLoader 爬取網頁並建立向量資料庫

import os
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 從 .env 載入環境變數（如果有的話）
load_dotenv()

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_web_nb")

print(f"持久化目錄: {persistent_directory}")

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量存儲...")

    # 步驟 1：使用 WebBaseLoader 爬取網頁內容
    # WebBaseLoader 載入網頁並提取其內容
    # 使用繁體中文網頁範例 - 維基百科台灣相關頁面
    urls = [
        "https://zh.wikipedia.org/zh-tw/台灣",
        "https://zh.wikipedia.org/zh-tw/臺北101"
    ]

    print("\n--- 正在爬取網頁內容 ---")
    print(f"目標網址: {urls}")
    
    # 建立網頁內容載入器
    loader = WebBaseLoader(urls)
    documents = loader.load()
    
    print(f"\n成功載入 {len(documents)} 個網頁")
    for i, doc in enumerate(documents, 1):
        print(f"  網頁 {i}: {doc.metadata.get('source', 'Unknown')}")
        print(f"  內容長度: {len(doc.page_content)} 字元")

    # 步驟 2：將爬取的內容分割成塊
    # CharacterTextSplitter 將文本分割成較小的塊
    # chunk_size=1000: 每個文本區塊最多 1000 個字元
    # chunk_overlap=200: 區塊之間重疊 200 個字元
    print("\n--- 正在分割文本 ---")
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print(f"文件塊數量: {len(docs)}")
    print(f"\n範例塊 (前 200 字元):")
    print(f"{docs[0].page_content[:200]}...\n")

    # 步驟 3：為文件塊建立嵌入
    # HuggingFaceEmbeddings 將文本轉換為捕捉語義意義的數值向量
    print("--- 正在建立嵌入 ---")
    embeddings = HuggingFaceEmbeddings(
        model_name="jinaai/jina-embeddings-v2-base-zh"
    )
    print("完成建立嵌入")

    # 步驟 4：使用嵌入建立並持久化向量存儲
    # Chroma 儲存嵌入以進行高效搜尋
    print("\n--- 正在建立向量存儲 ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory
    )
    print(f"完成建立向量存儲於 {persistent_directory}")

else:
    print("向量存儲已存在。無需初始化。")
    print("如需重新爬取網頁，請刪除資料庫目錄後重新執行。")

print("\n✅ 第1個儲存格執行完成")
print("\n💡 提示：")
print("- WebBaseLoader 可以爬取任何公開的網頁")
print("- 建議選擇內容豐富、結構清晰的繁體中文網頁")
print("- 注意遵守網站的 robots.txt 和使用條款")
print("- 對於需要登入或動態載入的網頁，可考慮使用 Firecrawl 或 Playwright")
print("\n📌 範例網頁說明：")
print("- 維基百科台灣頁面：介紹台灣的地理、歷史、文化等")
print("- 臺北101頁面：介紹台灣著名地標建築")

In [ ]:
# 第2個儲存格：查詢網頁內容並展示 RAG 應用

import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

# 從 .env 載入環境變數
load_dotenv()

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_web_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-zh"
)

# 使用嵌入函數載入現有的向量存儲
db = Chroma(
    persist_directory=persistent_directory,
    embedding_function=embeddings
)

# 步驟 1：定義使用者的問題（改為繁體中文問題）
query = "台灣的地理位置在哪裡？"

print(f"\n{'='*70}")
print(f"使用者問題: {query}")
print(f"{'='*70}")

# 步驟 2：根據查詢檢索相關文件
# 建立用於查詢向量存儲的檢索器
# search_type="similarity": 使用相似度搜尋
# k=3: 返回最相關的 3 個文件
print("\n--- 正在檢索相關文件 ---")
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)
relevant_docs = retriever.invoke(query)

# 顯示檢索到的文件
print(f"\n找到 {len(relevant_docs)} 個相關文件:\n")
for i, doc in enumerate(relevant_docs, 1):
    print(f"--- 文件 {i} ---")
    print(f"來源: {doc.metadata.get('source', 'Unknown')}")
    print(f"內容預覽: {doc.page_content[:200]}...\n")

# 步驟 3：整合檢索到的文件與 LLM
# 這是 RAG 的核心：將檢索到的文件作為上下文提供給 LLM
print("--- 正在準備 RAG 提示 ---")
combined_input = (
    "以下是一些可能有助於回答問題的網頁內容："
    + query
    + "\n\n相關內容：\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\n請僅根據提供的網頁內容提供答案。如果在內容中找不到答案，請回覆『我不確定』。"
)

print(f"\n合併後的提示長度: {len(combined_input)} 字元")

# 步驟 4：使用 LLM 生成答案
# 建立 ChatOllama 模型（使用本地 LLM）
print("\n--- 正在生成答案 ---")
model = ChatOllama(model="llama3.2")

# 定義模型的訊息
# SystemMessage: 設定 AI 的角色和行為
# HumanMessage: 使用者的問題加上檢索到的上下文
messages = [
    SystemMessage(content="你是一個有幫助的助手，專門回答關於台灣的問題。請用繁體中文回答。"),
    HumanMessage(content=combined_input),
]

# 使用合併的輸入調用模型
result = model.invoke(messages)

# 顯示生成的回應
print(f"\n{'='*70}")
print("AI 生成的回應:")
print(f"{'='*70}")
print(result.content)
print(f"{'='*70}")

print("\n--- RAG 流程總結 ---")
print("1️⃣  使用 WebBaseLoader 爬取網頁內容")
print("2️⃣  將內容分割並建立向量資料庫")
print("3️⃣  使用者提問")
print("4️⃣  向量檢索相關內容")
print("5️⃣  將檢索結果作為上下文提供給 LLM")
print("6️⃣  LLM 基於網頁內容生成答案")

print("\n💡 應用場景：")
print("- 知識庫問答系統（如：台灣旅遊資訊）")
print("- 新聞/部落格內容搜尋")
print("- 產品說明查詢")
print("- 教育資源整合")
print("\n📌 範例查詢建議：")
print("- 台灣的地理位置在哪裡？")
print("- 臺北101有多高？")
print("- 台灣有哪些特色文化？")

In [ ]:
# 第3個儲存格：使用 LCEL 建立 RAG Chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

# 建立 ChatOllama 模型
# 注意：模型已在第2個儲存格中建立，此處為保持儲存格獨立性而重新建立
model = ChatOllama(model="llama3.2")

# 建立提示模板
# 這個模板指導 LLM 如何使用從網頁爬取的上下文來回答問題
template = '''
僅根據以下從網頁爬取的上下文來回答問題：
{context}

問題：{question}
'''
prompt = ChatPromptTemplate.from_template(template)

# 格式化文件函數
# 將檢索到的文件列表轉換為單一字串
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 建立 RAG Chain
# 這是一個使用 LCEL (LangChain Expression Language) 的標準 RAG 實作
# retriever 已在第 2 個儲存格中定義
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 執行 RAG Chain
print("\n--- 使用 LCEL RAG Chain 執行 ---")
# 我們傳遞原始問題，Chain 會自動處理檢索、格式化和模型調用
# query 已在第 2 個儲存格中定義
response = rag_chain.invoke(query)

print(f"\n問題: {query}")
print(f"\nAI 回應:\n{response}")

print("\n--- LCEL RAG Chain 流程說明 ---")
print("1. RunnablePassthrough() 將使用者問題傳遞下去")
print("2. `retriever | format_docs` 檢索文件並格式化為字串")
print("3. `prompt` 將上下文和問題填入模板")
print("4. `model` 使用提示生成回應")
print("5. `StrOutputParser()` 提取回應內容")